In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd

In [4]:
# Importing Data
train=pd.read_csv('/content/drive/MyDrive/datasets/train.csv')

In [5]:
test=pd.read_csv('/content/drive/MyDrive/datasets/test.csv')

In [6]:
meal=pd.read_csv('/content/drive/MyDrive/datasets/meal_info.csv')

In [7]:
# Merging DataFrames
train=train.merge(meal, on='meal_id')


In [8]:
# Take inputs from the user
center_id = 55
meal_id = 1993

In [9]:
# Filter training data
train_df = train[train['center_id']==center_id]
train_df = train_df[train_df['meal_id']==meal_id]

In [10]:
# Preprocessing for time-series analysis
def pretime():

  df=train_df
  period = len(train_df)
  train_df['Date'] = pd.date_range('2015-01-08', periods=period, freq='W')
  train_df['Day'] = train_df['Date'].dt.day
  train_df['Month'] = train_df['Date'].dt.month
  train_df['Year'] = train_df['Date'].dt.year
  train_df['Quarter'] = train_df['Date'].dt.quarter


In [11]:
pretime()

In [12]:
train_df

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,category,cuisine,Date,Day,Month,Year,Quarter
11092,1466964,1,55,1993,136.83,135.83,0,0,270,Beverages,Thai,2015-01-11,11,1,2015,1
11169,1035555,2,55,1993,133.89,133.89,0,0,121,Beverages,Thai,2015-01-18,18,1,2015,1
11246,1282652,3,55,1993,134.86,133.86,0,0,258,Beverages,Thai,2015-01-25,25,1,2015,1
11323,1066744,4,55,1993,134.89,133.89,0,0,82,Beverages,Thai,2015-02-01,1,2,2015,1
11400,1115779,5,55,1993,147.50,145.50,0,0,81,Beverages,Thai,2015-02-08,8,2,2015,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21823,1210404,141,55,1993,149.41,148.41,0,0,134,Beverages,Thai,2017-09-17,17,9,2017,3
21900,1162047,142,55,1993,152.35,152.35,0,0,189,Beverages,Thai,2017-09-24,24,9,2017,3
21977,1006104,143,55,1993,151.35,150.35,0,0,109,Beverages,Thai,2017-10-01,1,10,2017,4
22054,1107848,144,55,1993,151.35,150.35,0,0,190,Beverages,Thai,2017-10-08,8,10,2017,4


LSTM

In [13]:
lstm_data = train_df.drop(columns=['id','center_id','meal_id','category','cuisine'])
lstm_data = lstm_data.set_index(['Date'])

In [14]:
period=len(train_df)

In [15]:
x_train = lstm_data.drop(columns='num_orders')
y_train = lstm_data['num_orders']
y_train = np.log1p(y_train)
split_size = period-15
X_train = x_train.iloc[:split_size,:]
X_test = x_train.iloc[split_size:,:]
Y_train = y_train.iloc[:split_size]
Y_test = y_train.iloc[split_size:]

In [16]:
lstm_data

,week,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,Day,Month,Year,Quarter
Date,,,,,,,,,,
2015-01-11,1,136.83,135.83,0,0,270,11,1,2015,1
2015-01-18,2,133.89,133.89,0,0,121,18,1,2015,1
2015-01-25,3,134.86,133.86,0,0,258,25,1,2015,1
2015-02-01,4,134.89,133.89,0,0,82,1,2,2015,1
2015-02-08,5,147.50,145.50,0,0,81,8,2,2015,1
...,...,...,...,...,...,...,...,...,...,...
2017-09-17,141,149.41,148.41,0,0,134,17,9,2017,3
2017-09-24,142,152.35,152.35,0,0,189,24,9,2017,3
2017-10-01,143,151.35,150.35,0,0,109,1,10,2017,4


LSTM (Light Model)

In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.optimizers import Adam
from sklearn.metrics import mean_squared_error

# Columns to train on
columns_to_train = ["week", "checkout_price", "base_price", "emailer_for_promotion", "homepage_featured", "Day", "Month", "Year", "Quarter"]

# Extract features and target variable
X = lstm_data[columns_to_train].values
y = lstm_data["num_orders"].values

# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape data for LSTM input (assuming a time series structure)
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

# Model
model = Sequential()
model.add(LSTM(100, input_shape=(X_train.shape[1], X_train.shape[2]), activation='relu', return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))

# Compilation
optimizer = Adam(learning_rate=0.001)
model.compile(loss='mse', optimizer=optimizer)

# Training
history = model.fit(x=X_train, y=y_train, epochs=200, batch_size=512, validation_data=(X_test, y_test))

# Print MSE
test_loss = model.evaluate(X_test, y_test, verbose=0)
print(f'Training MSE: {history.history["loss"][-1]}, Test MSE: {test_loss}')


Epoch 1/200
1/1 [==============================] - 5s 5s/step - loss: 82222.5625 - val_loss: 67146.5000
Epoch 2/200
1/1 [==============================] - 0s 37ms/step - loss: 82219.8594 - val_loss: 67143.7812
Epoch 3/200
1/1 [==============================] - 0s 34ms/step - loss: 82216.8594 - val_loss: 67141.0547
Epoch 4/200
1/1 [==============================] - 0s 31ms/step - loss: 82214.1094 - val_loss: 67138.3516
Epoch 5/200
1/1 [==============================] - 0s 31ms/step - loss: 82211.1797 - val_loss: 67135.6406
Epoch 6/200
1/1 [==============================] - 0s 32ms/step - loss: 82208.9453 - val_loss: 67132.9297
Epoch 7/200
1/1 [==============================] - 0s 32ms/step - loss: 82205.5938 - val_loss: 67130.1953
Epoch 8/200
1/1 [==============================] - 0s 33ms/step - loss: 82202.8359 - val_loss: 67127.3906
Epoch 9/200
1/1 [==============================] - 0s 30ms/step - loss: 82200.3672 - val_loss: 67124.5156
Epoch 10/200
1/1 [==============================

Bidirectional LSTM (Medium Model)

In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Bidirectional
from keras.optimizers import Adam
from sklearn.metrics import mean_squared_error

# Columns to train on
columns_to_train = ["week", "checkout_price", "base_price", "emailer_for_promotion", "homepage_featured", "Day", "Month", "Year", "Quarter"]

# Extract features and target variable
X = lstm_data[columns_to_train].values
y = lstm_data["num_orders"].values

# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape data for LSTM input (assuming a time series structure)
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

# Model
model = Sequential()
model.add(Bidirectional(LSTM(100, input_shape=(X_train.shape[1], X_train.shape[2]), activation='relu', return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(50, activation='relu')))
model.add(Dropout(0.2))
model.add(Dense(1))

# Compilation
optimizer = Adam(learning_rate=0.001)
model.compile(loss='mse', optimizer=optimizer)

# Training
history = model.fit(x=X_train, y=y_train, epochs=200, batch_size=512, validation_data=(X_test, y_test))

# Print MSE
test_loss = model.evaluate(X_test, y_test, verbose=0)
print(f'Training MSE: {history.history["loss"][-1]}, Test MSE: {test_loss}')


Epoch 1/200
1/1 [==============================] - 5s 5s/step - loss: 82214.3906 - val_loss: 67136.5547
Epoch 2/200
1/1 [==============================] - 0s 38ms/step - loss: 82210.2266 - val_loss: 67131.6641
Epoch 3/200
1/1 [==============================] - 0s 40ms/step - loss: 82205.0469 - val_loss: 67126.6094
Epoch 4/200
1/1 [==============================] - 0s 39ms/step - loss: 82200.2266 - val_loss: 67121.4375
Epoch 5/200
1/1 [==============================] - 0s 39ms/step - loss: 82193.0859 - val_loss: 67115.9531
Epoch 6/200
1/1 [==============================] - 0s 36ms/step - loss: 82186.2422 - val_loss: 67110.1406
Epoch 7/200
1/1 [==============================] - 0s 36ms/step - loss: 82183.8594 - val_loss: 67104.0078
Epoch 8/200
1/1 [==============================] - 0s 38ms/step - loss: 82177.1406 - val_loss: 67097.5156
Epoch 9/200
1/1 [==============================] - 0s 46ms/step - loss: 82167.5547 - val_loss: 67090.5781
Epoch 10/200
1/1 [==============================

 Bidirectional LSTM (Heavy Model)

In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Bidirectional, BatchNormalization
from keras.optimizers import Adam
from sklearn.metrics import mean_squared_error
from keras.callbacks import EarlyStopping

# Columns to train on
columns_to_train = ["week", "checkout_price", "base_price", "emailer_for_promotion", "homepage_featured", "Day", "Month", "Year", "Quarter"]

# Extract features and target variable
X = lstm_data[columns_to_train].values
y = lstm_data["num_orders"].values

# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape data for LSTM input (assuming a time series structure)
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

# Model
model = Sequential()
model.add(Bidirectional(LSTM(128, activation='relu', return_sequences=True), input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(64, activation='relu')))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))

# Compilation
optimizer = Adam(learning_rate=0.0005)
model.compile(loss='mse', optimizer=optimizer)

# Training with early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

history = model.fit(x=X_train, y=y_train, epochs=300, batch_size=512, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Calculate training MSE manually
train_preds = model.predict(X_train)
train_mse = mean_squared_error(y_train, train_preds)
print(f'Manually Calculated Training MSE: {train_mse}')

# Print MSE
test_loss = model.evaluate(X_test, y_test, verbose=0)
print(f'Training MSE: {history.history["loss"][-1]}, Test MSE: {test_loss}')


Epoch 1/300
1/1 [==============================] - 7s 7s/step - loss: 82286.2812 - val_loss: 67144.2969
Epoch 2/300
1/1 [==============================] - 0s 74ms/step - loss: 82235.2500 - val_loss: 67141.7266
Epoch 3/300
1/1 [==============================] - 0s 74ms/step - loss: 82188.3984 - val_loss: 67139.1875
Epoch 4/300
1/1 [==============================] - 0s 74ms/step - loss: 82143.4219 - val_loss: 67136.6562
Epoch 5/300
1/1 [==============================] - 0s 69ms/step - loss: 82098.6484 - val_loss: 67134.1250
Epoch 6/300
1/1 [==============================] - 0s 71ms/step - loss: 82053.8047 - val_loss: 67131.5703
Epoch 7/300
1/1 [==============================] - 0s 69ms/step - loss: 82009.6484 - val_loss: 67128.9766
Epoch 8/300
1/1 [==============================] - 0s 67ms/step - loss: 81964.8438 - val_loss: 67126.3359
Epoch 9/300
1/1 [==============================] - 0s 66ms/step - loss: 81917.2578 - val_loss: 67123.6562
Epoch 10/300
1/1 [==============================